In [1]:
import pandas as pd
import numpy as np
from urllib.parse import urlparse

In [8]:
ben = pd.read_csv("./benign-data.csv")
ot = pd.read_csv("./openpshish-data.csv")
pt = pd.read_csv("./phishtank-data.csv")
k = pd.read_csv("./kernun-data.csv")

In [9]:
ben.rename({'index': 'url'}, inplace=True, axis=1)

In [10]:
ben['host'] = ben.apply(lambda row: urlparse(row.url).netloc, axis=1)
ot['host'] = ot.apply(lambda row: urlparse(row.url).netloc, axis=1)
pt['host'] = pt.apply(lambda row: urlparse(row.url).netloc, axis=1)
k['host'] = k.apply(lambda row: urlparse(row.url).netloc, axis=1)

In [11]:
ben.drop_duplicates('host', inplace=True)
ot.drop_duplicates('host', inplace=True)
pt.drop_duplicates('host', inplace=True)
k.drop_duplicates('host', inplace=True)

In [12]:
asn_ok = ben['asn_value']
asn_ot = ot['asn_value']
asn_pt = pt['asn_value']
asn_k = k['asn_value']

In [13]:
asn_ok = asn_ok.append(asn_k)

In [15]:
asn_ok = asn_ok[asn_ok != '0']
asn_ok.size

27002

In [22]:
asn_ok.value_counts()

AS15169     2241
AS13335      697
AS43541      532
AS24971      497
AS197019     398
            ... 
AS7979         1
AS62083        1
AS52115        1
AS12895        1
AS20884        1
Name: asn_value, Length: 715, dtype: int64

In [24]:
asn_ot = asn_ot[asn_ot != '0']
asn_pt = asn_pt[asn_pt != '0']

In [25]:
asn_bad = asn_ot.append(asn_pt)
asn_bad

0       AS36459
1       AS31178
2       AS34119
3       AS54290
5       AS54290
         ...   
9911    AS15169
9912    AS15169
9913    AS15169
9914    AS29226
9915    AS15169
Name: asn_value, Length: 4158, dtype: object

In [26]:
asn_bad.value_counts()

AS15169     379
AS13335     369
AS46606     281
AS26496     278
AS204915    193
           ... 
AS26481       1
AS47268       1
AS199524      1
AS40386       1
AS3249        1
Name: asn_value, Length: 636, dtype: int64

In [27]:
df = pd.concat([asn_ok.value_counts(), asn_bad.value_counts()], axis=1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [28]:
df.columns = ['ok', 'phish']

In [32]:
df.loc[:, df.isna().any()]

,ok,phish
A,1.0,NaN
ADDRESSES,3.0,NaN
AS10310,2.0,3.0
AS10316,1.0,1.0
AS10565,1.0,NaN
...,...,...
as199391,NaN,1.0
as31377,1.0,NaN
as35913,NaN,1.0
as44976,NaN,1.0


In [33]:
df.fillna(value=0, inplace=True)

In [34]:
ok_sum = sum(df.ok)
phish_sum = sum(df.phish)

In [35]:
ok_sum

9531.0

In [36]:
phish_sum

4158.0

In [38]:
df.reset_index(inplace=True)

In [39]:
df.rename({'index': 'asn'}, axis=1, inplace=True)


In [40]:
df

,asn,ok,phish
0,A,1.0,0.0
1,ADDRESSES,3.0,0.0
2,AS10310,2.0,3.0
3,AS10316,1.0,1.0
4,AS10565,1.0,0.0
...,...,...,...
1162,as199391,0.0,1.0
1163,as31377,1.0,0.0
1164,as35913,0.0,1.0
1165,as44976,0.0,1.0


In [41]:
df['rogue_index'] = (df.phish / phish_sum) / ((df.phish / phish_sum) + (df.ok / ok_sum))

In [42]:
df.rogue_index.fillna(value=0, inplace=True)

In [43]:
df.asn = df.asn.str.upper()

In [44]:
df

,asn,ok,phish,rogue_index
0,A,1.0,0.0,0.000000
1,ADDRESSES,3.0,0.0,0.000000
2,AS10310,2.0,3.0,0.774689
3,AS10316,1.0,1.0,0.696252
4,AS10565,1.0,0.0,0.000000
...,...,...,...,...
1162,AS199391,0.0,1.0,1.000000
1163,AS31377,1.0,0.0,0.000000
1164,AS35913,0.0,1.0,1.000000
1165,AS44976,0.0,1.0,1.000000


In [45]:
df[['asn', 'rogue_index']].to_csv('./asn_rogue_index.csv')